In [45]:
import json
import pandas as pd

In [46]:
df = pd.read_json('result.json')

In [47]:
df = df.T
df.head()

,keyword,abstract,author,papername
0,"[Searchlights, Artificial Light, Surveillance,...",Searchlights have been used historically for a...,Claudia Arozqueta,"In the Spotlight: Searchlights, Art, Surveilla..."
1,"[Invisible animals, non-human animals, halluci...",My research-creation on invisible animals expl...,Donna Szoke,Invisible Animals
2,"[Self-representation, quantified self, mobile ...","This paper presents BeHAVE, a web-based audiov...",Sihwa Park,Multimodal Data Portrait for Representing Mobi...
3,"[Ecological surveillance, Biosensing technolog...",This paper explores merging technology and bio...,Zane Cerpina,Plant Based Bio-Drone for Environmental Monito...
4,"[Keywords: Bio-art, Schrödinger’s hypothesis o...",This paper introduces Korean bio-artist’s work...,"Jongcheon Shin, Siwon Lee, Suk Chon, Keyan Jia...",Seeing Life: The Impalpable Entanglement of an...


In [48]:
for idx, i in enumerate(df['keyword']):
    temp =[]
    for j in i:
        j = j.lower()
        j.replace("keywords: ","")
        temp.append(j)
    df.iloc[idx]['keyword'] = temp

In [49]:
df.head()

,keyword,abstract,author,papername
0,"[searchlights, artificial light, surveillance,...",Searchlights have been used historically for a...,Claudia Arozqueta,"In the Spotlight: Searchlights, Art, Surveilla..."
1,"[invisible animals, non-human animals, halluci...",My research-creation on invisible animals expl...,Donna Szoke,Invisible Animals
2,"[self-representation, quantified self, mobile ...","This paper presents BeHAVE, a web-based audiov...",Sihwa Park,Multimodal Data Portrait for Representing Mobi...
3,"[ecological surveillance, biosensing technolog...",This paper explores merging technology and bio...,Zane Cerpina,Plant Based Bio-Drone for Environmental Monito...
4,"[keywords: bio-art, schrödinger’s hypothesis o...",This paper introduces Korean bio-artist’s work...,"Jongcheon Shin, Siwon Lee, Suk Chon, Keyan Jia...",Seeing Life: The Impalpable Entanglement of an...


In [50]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    docs = docs.lower()  # Convert to lowercase.
    docs = tokenizer.tokenize(docs)  # Split into words.
    docs = [w for w in docs if not w in stopwords.words('english')]
    # Remove numbers, but not words that contain numbers.
    docs = [token for token in docs if not token.isdigit()]
    # Remove words that are only one character.
    docs = [token for token in docs if len(token) > 1]
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [lemmatizer.lemmatize(token) for token in docs]
  
    return docs

In [51]:
for idx, i in enumerate(df['abstract']):
    abstract = docs_preprocessor(i)
    df.iloc[idx]['abstract'] = abstract

In [52]:
df.head()

,keyword,abstract,author,papername
0,"[searchlights, artificial light, surveillance,...","[searchlight, used, historically, artistic, mi...",Claudia Arozqueta,"In the Spotlight: Searchlights, Art, Surveilla..."
1,"[invisible animals, non-human animals, halluci...","[research, creation, invisible, animal, explor...",Donna Szoke,Invisible Animals
2,"[self-representation, quantified self, mobile ...","[paper, present, behave, web, based, audiovisu...",Sihwa Park,Multimodal Data Portrait for Representing Mobi...
3,"[ecological surveillance, biosensing technolog...","[paper, explores, merging, technology, biology...",Zane Cerpina,Plant Based Bio-Drone for Environmental Monito...
4,"[keywords: bio-art, schrödinger’s hypothesis o...","[paper, introduces, korean, bio, artist, work,...","Jongcheon Shin, Siwon Lee, Suk Chon, Keyan Jia...",Seeing Life: The Impalpable Entanglement of an...


In [53]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [54]:
zipdata = zip(df['author'],df['abstract'])

In [55]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in zipdata]

In [56]:
tagged_data[1]

TaggedDocument(words=['research', 'creation', 'invisible', 'animal', 'explores', 'invisible', 'visual', 'realm', 'order', 'explore', 'immanence', 'power', 'non', 'visual', 'knowledge', 'utilizing', 'digital', 'technology', 'create', 'medium', 'artwork', 'think', 'transformational', 'object', 'object', 'shift', 'u', 'new', 'way', 'perceiving', 'leap', 'perception', 'medium', 'art', 'experience', 'change', 'understanding', 'world', 'challenging', 'notion', 'utility', 'animal', 'function', 'technology', 'work', 'offer', 'ethic', 'care', 'liberation', 'instrumental', 'rationalism'], tags=['Donna Szoke'])

In [57]:
import re 
import gensim
from gensim.models import doc2vec
from collections import namedtuple
from random import shuffle

In [58]:
model = gensim.models.Doc2Vec(vector_size=50, window=10, min_count=1, workers=8, alpha=0.025, min_alpha=0.015, 
                              epochs=500)

#sample=1e-4, negative=5,

#shuffling is better (ot needed at each trianing epoch
shuffle(tagged_data)
#Build vocabulary from a sequence of sentences 
model.build_vocab(tagged_data)
#Update the model’s neural weights from a sequence of sentences
model.train(tagged_data, epochs=model.epochs, total_examples=model.corpus_count)

In [59]:
model.docvecs.most_similar('Claudia Arozqueta', topn=101)

[('Janna Ahrndt', 0.6277286410331726),
 ('Honghao Deng, Jiabao Li, Xuesong Zhang & Panagiotis Michalatos',
  0.6229113340377808),
 ('Sara Bonaventura', 0.620789110660553),
 ('Lingdong Huang, Zheng Jiang, Syuan-Cheng Sun, Tong Bai, Eunsu Kang & Barnabas Poczos',
  0.6007415652275085),
 ('Kyungho Lee', 0.5934253334999084),
 ('Irene Eunyoung Lee', 0.592743992805481),
 ('Tobias Klein & Harald Kraemer', 0.5686616897583008),
 ('Young Ho Kim, Yang Kyu Lim & Jin Wan Park', 0.5631190538406372),
 ('Scott Rettberg', 0.5541275143623352),
 ('Claudia Costa Pederson', 0.5289748907089233),
 ('Intae Hwang & Alenda Y. Chang', 0.5232059955596924),
 ('Aleksandra Dulic & Miles Thorogood', 0.508289098739624),
 ('Paula Andrea Escandón, Andrés Felipe Roldán & Fernando Luna',
  0.4979220926761627),
 ('Jason Kennedy', 0.4967525005340576),
 ('Haoyi Zhang', 0.49611949920654297),
 ('Masayuki Akamatsu, Yasuko Imura, Tomoki Kobayashi, Iku Harada & Shigeru Matsui',
  0.4944039583206177),
 ('Ian Willcock', 0.493512958

In [60]:
col = model.docvecs.index2entity
len(col)

101

In [61]:
author2author =[]
for i in col:
    for n,v in model.docvecs.most_similar(i, topn=101):
        if v > 0.5:
            documents = {}
            documents['source'] = i
            documents['target'] = n
            author2author.append(documents)

In [62]:
author2author[0:5]

[{'source': 'Meredith Drum', 'target': 'Irene Eunyoung Lee'},
 {'source': 'Meredith Drum', 'target': 'Kyungho Lee'},
 {'source': 'Meredith Drum', 'target': 'Anna Madeleine Raupach'},
 {'source': 'Meredith Drum', 'target': 'Ian Willcock'},
 {'source': 'Meredith Drum', 'target': 'Minso Kim'}]

In [63]:
author2keyword = []
for i in range(len(df)):
    for j in df.iloc[i]['keyword']:
        documents = {}
        documents['source'] = df.iloc[i]['author']
        documents['target'] = j.replace('keywords: ','')
        author2keyword.append(documents)

In [64]:
author2keyword[0:5]

[{'source': 'Claudia Arozqueta', 'target': 'searchlights'},
 {'source': 'Claudia Arozqueta', 'target': 'artificial light'},
 {'source': 'Claudia Arozqueta', 'target': 'surveillance'},
 {'source': 'Claudia Arozqueta', 'target': 'installation art'},
 {'source': 'Claudia Arozqueta', 'target': 'memorial art'}]

In [65]:
node = []
for i in range(len(df)):
    documents = {}
    documents['id'] = df.iloc[i]['author']
    documents['label'] = 'author'
    node.append(documents)
    for j in df.iloc[i]['keyword']:
        keydoc = {}
        keydoc['id'] = j
        keydoc['label'] = 'keyword'
        node.append(keydoc)

In [66]:
node[0:5]

[{'id': 'Claudia Arozqueta', 'label': 'author'},
 {'id': 'searchlights', 'label': 'keyword'},
 {'id': 'artificial light', 'label': 'keyword'},
 {'id': 'surveillance', 'label': 'keyword'},
 {'id': 'installation art', 'label': 'keyword'}]

In [68]:
with open('author2author.json', 'w') as f:
    json.dump(author2author , f, indent='\t')
with open('author2keyword.json', 'w') as f:
    json.dump(author2keyword , f, indent='\t')
with open('node.json', 'w') as f:
    json.dump(node , f, indent='\t')